In [1]:
import sys
import pandas as pd
import torch
from torch import nn

# Pandas: Reading the data
df = pd.read_csv("used_cars.csv")

# Pandas: Preparing the data
age = df["model_year"].max() - df["model_year"]

milage = df["milage"]
milage = milage.str.replace(",", "")
milage = milage.str.replace(" mi.", "")
milage = milage.astype(int)

# TODO: Create an 'accident_free' feature where value is 1 if df["accident"] == "None reported", else 0

price = df["price"]
price = price.str.replace("$", "")
price = price.str.replace(",", "")
price = price.astype(int)

# Torch: Creating X and y data (as tensors)
X = torch.column_stack([
    # TODO: Add accident_free as the first column
    torch.tensor(age, dtype=torch.float32),
    torch.tensor(milage, dtype=torch.float32)
])
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

y = torch.tensor(price, dtype=torch.float32).reshape((-1, 1))
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std
# sys.exit()

model = nn.Linear(2, 1)  # TODO: Change input dimension from 2 to 3
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for i in range(0, 10000):
    # Training pass
    optimizer.zero_grad()
    outputs = model(X)
    loss = loss_fn(outputs, y)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss)

X_data_accident = torch.tensor([
    # TODO: Add a leading 0/1 for accident_free in each row
    [5, 10000],
    [2, 10000],
    [5, 20000]
], dtype=torch.float32)

with torch.no_grad():
    prediction_accident = model((X_data_accident - X_mean) / X_std)
    print(prediction_accident * y_std + y_mean)